In [ ]:
import os
import shutil
import random
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Reshape,  \
                              GlobalAveragePooling2D, AveragePooling2D, Input, Concatenate, Layer, MaxPool2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, Callback
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

In [ ]:
# gpu 준비
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2022-05-28 20:09:51.504447: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 20:09:51.666500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 20:09:51.667363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


1 Physical GPUs, 1 Logical GPUs


2022-05-28 20:09:51.676598: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-28 20:09:51.676954: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 20:09:51.677740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-28 20:09:51.678382: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login(key=wandb_api_key)
wandb.init(project="fer-torchvision-keras")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hhan14. Use `wandb login --relogin` to force relogin


In [ ]:
!tar xzvf ../input/challenges-in-representation-learning-facial-expression-recognition-challenge/fer2013.tar.gz
%mkdir ./models

fer2013/fer2013.csv
fer2013/README
fer2013/fer2013.bib
fer2013/


In [ ]:
DATA_PATH = "./fer2013/fer2013.csv"

In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5e028c07-beab-aa4b-1d93-73c1f1b1f681)


In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

2022-05-28 20:10:06.168397: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [ ]:
# 참조: https://minimin2.tistory.com/100
# 매 epoch 마다 data augmentation을 진행하기 위한 dataloader class

class Dataloader(Sequence):
    def __init__(self, x_set, y_set, transform=None, batch_size=64, shuffle=True):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.transform = transform # data augmentation 진행할 torchvision transform 모듈
        self.on_epoch_end()
    
    def img_preprocess(self, idx):
        img = np.array(self.x[idx])
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        label = np.array([self.y[idx]] * 10)
        return img, label

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    # 데이터를 batch size 만큼 불러오며 img_preprocess 통해 augmentation 한 데이터 반환
    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        batch_x_list, batch_y_list = map(list, zip(*[self.img_preprocess(i) for i in indices]))
        
        batch_x = np.array(batch_x_list)
        batch_y = np.array(batch_y_list)
        
        bs, ncrops, h, w, c = batch_x.shape
        batch_x = batch_x.reshape([-1, h, w, c])
        
        bs, ncrops, labels = batch_y.shape
        batch_y = batch_y.reshape([-1, labels])

        return batch_x, batch_y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

In [ ]:
# 참조: https://github.com/usef-kh/fer/blob/master/data/fer2013.py
def load_data(path=DATA_PATH):
    fer2013 = pd.read_csv(path)
    emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

    return fer2013, emotion_mapping

# df에서 이미지와 label을 추출하여 array로 반환
def prepare_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    onehot_encoder = OneHotEncoder()
    image_label = image_label.reshape(-1, 1)
    image_label = onehot_encoder.fit_transform(image_label)
    image_label = image_label.toarray()

    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image

    return image_array, image_label


def get_dataloaders(path=DATA_PATH, bs=64, augment=True):
    fer2013, emotion_mapping = load_data(path)

    xtrain, ytrain = prepare_data(fer2013[fer2013['Usage'] == 'Training'])
    xval, yval = prepare_data(fer2013[fer2013['Usage'] == 'PrivateTest'])
    xtest, ytest = prepare_data(fer2013[fer2013['Usage'] == 'PublicTest'])

    mu, st = 0, 255

    test_transform = transforms.Compose([
        transforms.TenCrop(40),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors]).permute(0, 2, 3, 1).numpy()),
    ])

    if augment:
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(48, scale=(0.8, 1.2)), # randomly rescale
            transforms.RandomApply([transforms.RandomAffine(0, translate=(0.2, 0.2))], p=0.5), # randomly translate
            transforms.RandomHorizontalFlip(),# randomly horizontal flip
            transforms.RandomApply([transforms.RandomRotation(10)], p=0.5), # randomly rotate
            transforms.TenCrop(40), # ten-crop
            transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
            transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors])), # normalize
            transforms.Lambda(lambda tensors: torch.stack([transforms.RandomErasing(p=0.5)(t) for t in tensors]).permute(0, 2, 3, 1).numpy()), # keras 모델의 입력인 (batch size, 40, 40, 1) input에 맞도록 reshape
        ])
    else:
        train_transform = test_transform

    trainloader = Dataloader(xtrain, ytrain, transform=train_transform, batch_size=128)
    valloader = Dataloader(xval, yval, transform=test_transform, batch_size=128)
    testloader = Dataloader(xtest, ytest, transform=test_transform, shuffle=False, batch_size=128)

    return trainloader, valloader, testloader

# 후에 정확도 측정 및 confusion matrix 생성을 위해 test 데이터 label만 불러오는 메서드
def get_test_labels():
    fer2013, emotion_mapping = load_data(DATA_PATH)
    x_test, y_test = prepare_data(fer2013[fer2013['Usage'] == 'PublicTest'])
    
    mu, st = 0, 255

    test_transform = transforms.Compose([
        transforms.TenCrop(40),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda tensors: torch.stack([transforms.Normalize(mean=(mu,), std=(st,))(t) for t in tensors]).permute(0, 2, 3, 1).numpy()),
    ])

    testloader_cm = Dataloader(x_test, y_test, batch_size=3589, transform=test_transform, shuffle=False)
    xtest, ytest = next(iter(testloader_cm))
    
    return emotion_mapping, np.argmax(ytest, axis=1) 

In [ ]:
trainloader, valloader, testloader = get_dataloaders(bs=128)

In [ ]:
# 참조: https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1

class AccHistory(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.lr = []
        
    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('accuracy'))
        self.lr.append(step_decay(len(self.acc)))
        print('lr:', step_decay(len(self.acc)))

def step_decay(epoch):
    initial_lrate = 0.1
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch)/epochs_drop))
    return lrate

acc_history = AccHistory()
lrate = LearningRateScheduler(step_decay)

In [ ]:
#참조 : https://bskyvision.com/504
#VGGNET을 논문에 맞게 BatchNormalization추가 및 변형 
#VGGNET은 ReLU 함수를 이용하고 네트워크의 깊이를 깊게 하고 parameter의 수를 적게하기 위해 모든 convolution layer에서의 kernel size는 3x3, stride는 1로 통일합니다. 
model = Sequential()
model.add(Conv2D(input_shape=(40, 40, 1), filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu")) #40x40x64의 feature map 생성
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), padding="valid")) #20x20x64로 feature map size 감소 

model.add(Conv2D(128, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu")) #20x20x128의 feature map 생성
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), padding="valid")) #10x10x128로 feature map size 감소

model.add(Conv2D(256, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu")) #10x10x256의 feature map 생성
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), padding="valid"))  #5x5x256로 feature map size 감소 

model.add(Conv2D(512, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu")) #5x5x512의 feature map 생성 
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size=(3,3), padding="same", strides=(1,1), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), padding="valid")) #2x2x512로 feature map size 감소

model.add(Flatten())  #feature map을 1차원으로 flatten -> (0,2048)
model.add(Dense(units=4096, activation="relu"))  #2048과 fully connected 되며 4096의 output
model.add(Dropout(0.2))
model.add(Dense(units=4096, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(7, activation="softmax")) #7개의 class를 분류하기 때문에 7개의 뉴런으로 구성.

decay_steps = 225 * 150 # train data // batch size * epoch
# lr을 코사인 함수의 형태로 주기적으로 스케줄링
lr = CosineDecay(initial_learning_rate=0.01, decay_steps=decay_steps)
# Stochastic Gradient Descent(확률적 경사 하강법) 이용
opt = SGD(learning_rate=lr, momentum=0.9, nesterov=True, decay=0.0001)

model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 40, 40, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 40, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 40, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 20, 128)       5

In [ ]:
acc_model_path = 'models/best_acc_model.h5'

acc_checkpoint = ModelCheckpoint(filepath=acc_model_path, monitor='val_accuracy', save_best_only=True, save_weights_only=False, save_freq="epoch")
early = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='auto')

wandb.config = {
  "epochs": 150,
  "batch_size": 128
}

hist = model.fit(
    trainloader,
    validation_data=valloader,
    epochs=150,
    batch_size=128,
    callbacks=[acc_checkpoint, early, WandbCallback()]
)

wandb.finish()

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2022-05-28 20:10:18.393135: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150


2022-05-28 20:10:22.070090: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


225/225 [==============================] - 144s 586ms/step - loss: 1.9581 - accuracy: 0.2608 - val_loss: 1.8817 - val_accuracy: 0.1775
Epoch 2/150
225/225 [==============================] - 128s 568ms/step - loss: 1.6329 - accuracy: 0.3494 - val_loss: 1.7364 - val_accuracy: 0.3155
Epoch 3/150
225/225 [==============================] - 129s 572ms/step - loss: 1.4767 - accuracy: 0.4255 - val_loss: 1.3508 - val_accuracy: 0.4809
Epoch 4/150
225/225 [==============================] - 139s 616ms/step - loss: 1.3686 - accuracy: 0.4724 - val_loss: 1.2763 - val_accuracy: 0.5194
Epoch 5/150
225/225 [==============================] - 142s 632ms/step - loss: 1.2895 - accuracy: 0.5039 - val_loss: 1.1315 - val_accuracy: 0.5668
Epoch 6/150
225/225 [==============================] - 143s 633ms/step - loss: 1.2293 - accuracy: 0.5308 - val_loss: 1.1446 - val_accuracy: 0.5666
Epoch 7/150
225/225 [==============================] - 139s 618ms/step - loss: 1.1921 - accuracy: 0.5456 - val_loss: 1.1589 - val_

accuracy,▁▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▇▇▇▇▇█▇█████████████████████████████
val_loss,█▄▃▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅
accuracy,0.87487
best_epoch,24
best_val_loss,0.89684
epoch,99
loss,0.34028
val_accuracy,0.69607


In [ ]:
hist = model.evaluate(testloader)
print(hist)

29/29 [==============================] - 8s 260ms/step - loss: 1.4798 - accuracy: 0.6902
[1.4797916412353516, 0.6902479529380798]
